## Building 3-layer neural network from numpy
Let’s start by importing some libraires required for creating our neural network.

In [7]:
from __future__ import print_function
import numpy as np ## Fr numerical python
np.random.seed(42)

Every layer will have a forward pass and backpass implementation. let's create a mian class layer which can do a forward pass .*forward()* and backward pass .*backward()*.

In [8]:
class Layer:
    # A building block. Each layer is capable of performing two things:
    # - process inpput to get output 
    # output = layer.forward(input)
    
    # - propagate gradients through itself:
    # grad_input = layer.backward(input, gradoutput)
    
    # Some layers also have learnable parameters which they update 
    # during layer.backward.
    
    def __init__(self):
        # here we can initialize layer parameters (if any) 
        # and auxiliary stuff
        # A dummy layer does nothing
        pass
    def forward(self, input_):
        # Takes input data of shape [batch, input_units], 
        #return ouput data [batch, output_units]
        
        # A dummy layer just returns whatever it gets as input.
        return input_
    def backward(self, input_, grad_output):
        # Performs a backpropagation step through the layer, 
        # with respect to the given input.
        # To compute loss gradients w.r.t input, we need to apply 
        # chain rule (backprop):
        # d_loss/d_x = (d_loss/d_layer) * (d_layer/d_x)
        # luckily, we already receive d_loss/d_layer as input, 
        # so you only need to multiply it by d_layere/d_x.
        # If our layer has parameters (e.g. dense layer), 
        # we also need to update them here using d_loss/d_layer
        # Thegradient of dummy layer is precisely grad_output, 
        # but we'll write it more explicity
        num_units = input_.shape[1]
        d_layer_d_input = np.eye(num_units)
        return np.dot(grad_output, d_layer_d_input) # chain rule
    

## Nonlinearity ReLU layer
This is the simplest layer you can get: it simply applies a nonlinarity to each element of your network

In [9]:
class ReLU(Layer):
    def __init__(self):
        # ReLu layer simply applies elementwise rectified linear unit 
        # to all inputs
        pass
    
    def forward(self, input_):
        # Apply elementwise ReLU to [batch, input_units] matrix
        relu_forward = np.maximum(0, input_)
        return relu_forward
    
    def backward(self, input, grad_output):
        # Compute gradient of loss w.r.t ReLU input
        relu_grad = input_ > 0
        return grad_output*relu_grad


## Dense layer
Now let's build somethign more complicate. Unlike nonlinearity, a dense layer actully has something to learn. <br>
A dense layer applies affine transformation. in a vectorized form, it can be described as:
$$f(\mathbf{X}) = \mathbf{W} . \mathbf{X} + \mathbf{b}$$
Where:<br>
- $\mathbf{X}$ is an object-feature matrix of shape [batch_size, num_features], <br>
- $\mathbf{W}$ is a weight matrix [num_features, num_outputs]<br>
- and $\mathbf{b}$ is a vector of num_outputs biases.


Both $\mathbf{W}$ and $\mathbf{b}$ are initialized during layer creation and updated each time backward is called. Note that we are using **Xavier initialization** which is a trick to train our model to converge faster. Instead of initializing our weights with small numbers which are distributed randomly, we initialize our weights with mean zero and variance of 2/(number of inputs + number of outputs)

In [10]:
class Dense(Layer):
    
    def __init__(self, input_units, output_units, learning_rate=0.1):
        # A dense layer is a layer which performs a learned affine 
        # transformation:
        # f(X) = W*X + b
        
        self.learning_rate = learning_rate
        self.weights = np.random.normal(
            loc=0.0, 
            scale=np.sqrt(2/(input_units+output_units)),
            size=(input_units, output_units)
        )
        self.biases = np.zeros(output_units)
    
    def forward(self, input_):
        # perform an affine transformation:
        # f(X) = W*X + b
        
        # input shape: [batch, input_units]
        # output shape: [batch, output_units]
        
        return np.dot(input_, self.weights) + self.biases
    
    def backward(self, input_, grad_output):
        # compute d_f/d_x = d_f/d_dense * d_dense/d_x
        # where d_dense/d_x = weights transposed
        # and d_f/d_dense = grad_output
        
        grad_input = np.dot(grad_output, self.weights.T)
        
        # compute gradient w.r.t. weights and bias (things to update)
        grad_weights = np.dot(input_.T, grad_output)
        grad_biases = grad_output.mean(axis=0)*input_.shape[0]
        
        assert grad_weights.shape == self.weights.shape\
               and grad_biases == self.biases.shape
        
        # Here we perform a schochastic gradient descent step.
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases
        
        return grad_input     

## The loss function
Since we want to predict probabilities, it would be logical for us to define softmax nonlinearity on top of our network and compute loss given predicted probabilities. However, there is a better way to do so.

If we write down the expression for crossentropy as a function of softmax logits(a), you'll see:
$$loss = -\log\frac{e^{\alpha_{\text{correct}}}}{\sum_i{e^{\alpha_i}}}$$
If we take a closer look, we'll see that it can be written as:
$$loss = -\alpha_{\text{correct}}+\log\sum_i{e^{\alpha_i}}$$
It's called Log-softmax and it's better than naive log(softmax(a)) in all aspects:

- Better numerical stability
- Easier to get derivative rigt
- Marginally faster to compute

So why not just use log-softmax throughout our computation and never actually bother to estimate probabilities.